In [13]:
import numpy as np
import zarr
import os

from pycromanager import Bridge
from pycromanager import Studio; 

construct java object

In [2]:
from pycromanager import Core

core = Core()
print(core)

In [4]:
studio = Studio()
bridge = Bridge()
mm = bridge.get_studio()
#ds = mm.displays().get_active_data_viewer().get_data_provider()

C:\Users\lucil\AppData\Local\Temp\ipykernel_31216\3220403713.py:3: DeprecationWarning: Deprecated. use "from pycromanager import Studio; studio = Studio()"
  mm = bridge.get_studio()


parameters

In [26]:
nb_tp = 500  # number of time points
nb_ch = 2  # number of channels
nz = 600  # number of z slices
ny = 1024  # number of y pixels in each image
nx = 1536  # number of x pixels in each image
save_to = r"C:\Users\lucil\OneDrive\Bureau\DeMarco_Lab\fibse\zarr folder\data.zarr"  # save to this directory

create zarr folder

In [27]:
if not os.path.isdir(save_to):
    # create new zarr folder
    root = zarr.open(save_to, mode="w")
    ch0 = root.zeros(
        "ch0", shape=(nb_tp, nz, ny, nx), chunks=(1, 64, 512, 512), dtype="i2"
    )
    ch1 = root.zeros(
        "ch1", shape=(nb_ch, nz, ny, nx), chunks=(1, 64, 512, 512), dtype="i2"
    )
    print(root.tree())

/
 ├── ch0 (500, 600, 1024, 1536) int16
 └── ch1 (2, 600, 1024, 1536) int16


In [28]:
# open the zarr folder to save data into it
root = zarr.open(save_to, mode="rw")
ch0 = root["ch0"]
ch1 = root["ch1"]

In [ ]:
data_stack = np.zeros((nz, ny, nx))  # initialize nparray to store a z-stack
for t in range(nb_tp):
    for ch in range(nb_ch):
        for z in range(nz):
            coords_string = "t={0},p=0,c={1},z={2}".format(t, ch, z)
            coords = mm.data().create_coords(coords_string)
            mimg = ds.get_image(coords)
            data_stack[z, :, :] = np.reshape(
                mimg.get_raw_pixels(), newshape=[mimg.get_height(), mimg.get_width()]
            )
        if ch == 0:
            ch0[t, :, :, :] = data_stack
        else:
            ch1[t, :, :, :] = data_stack

bridge.close()